# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
df = pd.read_csv('data.csv')
display(df.head())
df.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В датасете 21525 строк и 12 столбцов. В столбцах days_employed и total_income есть пропуски. Столбцы children, days_employed, dob_years, total_income содержат количественные значения, столбцы education,	education_id, family_status, family_status_id, gender, income_type,	debt и purpose - категориальные. В столбце 'days_employed'(общий трудовой стаж в днях) встречаются отрицательные значения, а некоторые значения, если перевести дни в годы значительно превышают возраст клиентов банка. Учитывая неправдоподобность данных в столбце 'days_employed', а также отсутствие очевидной закономерности в ошибках в данных, можно сделать вывод, что данные не пригодны для анализа - однако, если этого требует задание, можно попробовать перевести отрицательные значения в положительные, а для значений стажа, которые превышают возраст, сдвигать зяпятую в дроби, пока стаж не станет меньше возраста.  В столбце purpose много неявных дубликатов, т.к. одна и та же цель сформулирована по-разному

## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
#display(df[df['days_employed'].isna()].head()) #как правило, пропуски в столбцах days_employed и total_income совпадают
#display(len(df[df['days_employed'].isna()])) # их 2174, поэтому лучше не выбрасывать, а заполнять.
# Сначала нужно разобраться с некорректностью данных в столбце days_employed:
df['days_employed'] = df['days_employed'].apply(abs) # сделаем из отрицательных значений положительные
# попытаемся разделить неправдоподобные значения столбца days_employed на 100
df.loc[df.loc[:,'days_employed'] / 365 > df['dob_years'], 'days_employed'] = df.loc[df.loc[:,'days_employed'] / 365 > df['dob_years']]['days_employed'] / 100
#display(df.loc[df.loc[:,'days_employed'] / 365 > df['dob_years']]) # проверим себя, код работает правильно, но обнаружена ещё одна проблема в данных - у некоторых представителей выборки возраст = 0
#учитывая, что ни столбец "days_employed", ни "dob_years", ни "income_type" в анализе не учавствуют, можно смело заполнять пропуски средними значениями
# ну и заполним пропуски средними значениями по типу занятости
mean_values = df.groupby('income_type').mean() #таблица со средними значениями
mean_values = mean_values.reset_index() # столбец income_type стал индексом, нужно исправить
income_types = list(df['income_type'].unique()) # список вариантов типов занятостей

for elem in income_types:
    filler = float(mean_values[mean_values['income_type'] == elem]['total_income'])
    to_fill = df.loc[df['income_type'] == elem, 'total_income']
    df.loc[df['income_type'] == elem, 'total_income'] = to_fill.fillna(filler)
# теперь сделаем такой же цикл для days_employed
for elem in income_types:
    filler = float(mean_values[mean_values['income_type'] == elem]['days_employed'])
    to_fill = df.loc[df['income_type'] == elem, 'days_employed']
    df.loc[df['income_type'] == elem, 'days_employed'] = to_fill.fillna(filler)
    
# display(df[df['dob_years'] == 0]) а ещё тут у 101 человека возраст = 0 лет
for elem in income_types:
    filler = float(mean_values[mean_values['income_type'] == elem]['dob_years'])
    df.loc[(df['income_type'] == elem) & (df['dob_years'] == 0), 'dob_years'] = filler
    
# количество детей в некоторых строках отрицательное, это тоже можно исправить через модуль
#display(df[df['children'] < 0])
df['children'] = df['children'].apply(abs) 


**Вывод**

Пропуски есть в двух столбцах: days_employed и total_income, их 2174, кроме того данные в столбце days_employed настолько неправдоподобны, что лучше бы его совсем убрать из таблицы и не анализировать. С другой стороны, в поставленной задаче и не требуется использовать их для ответа на ключевые вопросы, поэтому я попробовала привести их в правдоподобный вид. Отрицательные значения превратились в положительные с помощью .apply(abs). Одна из возможных причин невероятно высоких значений трудового стажа - сдвинутая запятая в дробях, примерно на 2 позиции. Чтобы вычленить эти "выбросы" из датафрейма, я применила фильтр "df.loc[:,'days_employed'] / 365 > df['dob_years']", т.е. выделила строки, где стаж в годах превышает возраст, а затем разделила полученные значения на 100. 
Затем заполнила пропуски средними зачениями по каждой из "income_type"

### Замена типа данных

In [4]:
df.info() # посмотрим на типы данных в столбцах

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 2.0+ MB


**Вывод**

Все числовые данные представлены целыми числами или числами с плавающей точкой, а все объекты - объектами, ничего менять не нужно. Если бы такая необходимость была, я бы воспользовалась методом .astype('int'). Некоторые значения float можно округлить до целых чисел и тогда будет использован .apply(round)

### Обработка дубликатов

In [5]:
# посмотрим на список уникальных значений столбца "education"
df['education'].value_counts()
# избавимся от дубликатов
df['education'] = df['education'].str.lower()
# проверим себя
df['education'].value_counts()
# теперь "family_status"
display(df['family_status'].value_counts())
# здесь дубликатов нет
df['gender'].value_counts()
df[df['gender'] == 'XNA'] # ну, пусть будет
# нужно удалить явные дубликаты
df = df.drop_duplicates().reset_index(drop=True) 

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

**Вывод**

Привела к нижнему регистру значения столбца "education", возникновение этих дубликатов вполне естественно, если в поле ввода не предусмотрено автоматическое приведение к нижнему регистру. В остальных столбцах типа "объект" дубликатов нет. Нашёлся один пропуск в графе "gender", но учитывая существование людей, не индентифицирующих свою половую принадлежность, наверное можно оставить его как есть. 

In [6]:
df.duplicated().sum()

0

### Лемматизация

In [7]:
df['purpose'].value_counts() #выведем уникальные значения столбца 'purpose'
from pymystem3 import Mystem # импорт библиотеки
m = Mystem() 

def purpose_lemmas(purpose): # напишем функцию, которая из заданных значений будет возвращать леммы через пробел
    purpose = ' '.join(m.lemmatize(purpose))
    return(purpose)

df['lemmas'] = df['purpose'].apply(purpose_lemmas) #применим эту функцию к столбцу "purpose"

display(df['lemmas'].unique()) # посмотрим на леммы в столбце

array(['покупка   жилье \n', 'приобретение   автомобиль \n',
       'дополнительный   образование \n', 'сыграть   свадьба \n',
       'операция   с   жилье \n', 'образование \n',
       'на   проведение   свадьба \n', 'покупка   жилье   для   семья \n',
       'покупка   недвижимость \n',
       'покупка   коммерческий   недвижимость \n',
       'покупка   жилой   недвижимость \n',
       'строительство   собственный   недвижимость \n', 'недвижимость \n',
       'строительство   недвижимость \n',
       'на   покупка   подержать   автомобиль \n',
       'на   покупка   свой   автомобиль \n',
       'операция   с   коммерческий   недвижимость \n',
       'строительство   жилой   недвижимость \n', 'жилье \n',
       'операция   со   свой   недвижимость \n', 'автомобиль \n',
       'заниматься   образование \n',
       'сделка   с   подержанный   автомобиль \n',
       'получение   образование \n', 'свадьба \n',
       'получение   дополнительный   образование \n',
       'покупка   свой 

**Вывод**

После выполнения лемматизации начинают определяться категории целей кредитов: жилье и недвижимость, автомобиль, образование, свадьба, строительство

### Категоризация данных

In [8]:
#напишем функцию для категоризации целей кредитов
def purpose_category(purpose):
    if 'жилье' in purpose or 'недвижимость' in purpose:
        return 'жилье и недвижимость'
    if 'образование' in purpose:
        return 'образование'
    if 'свадьба' in purpose:
        return 'свадьба'
    if 'автомобиль' in purpose or 'машина' in purpose:
        return 'автомобиль'
    else:
        return 'не определена'
df['purpose_group'] = df['lemmas'].apply(purpose_category)
display(df['purpose_group'].value_counts())
#как позже выяснилось, уровень дохода тоже удобнее категоризировать
#для начала, нужно понять в каком диапазоне разброс значений
max_income = round(df['total_income'].max())
min_income = round(df['total_income'].min())
mean_income = round(df['total_income'].mean())
median_income = round(df['total_income'].median())

print(max_income, "- максимум", min_income, "- минимум,", mean_income, "- среднее,", median_income, "- медиана.")
#чтобы относительно равномерно выделить 4 категории, попробуем разделить медиану пополам, 
#print(median_income / 2)
#print((max_income - median_income) / 2)#а затем найти середину расстояния между медианой и максимумом
#округлим значения и получится: 1) до 75 000; 2) от 75 000 до 150 000; 3) от 150 000 до 1 000 000; 4) 1 000 000 и более
#попробуем разбить данные на эти категории. В категории более 1 000 000 получается всего 25 значений, 
# поэтому принято решение опустить верхнее значение до 750 000
def income_category(income):
    if income < 75000:
        return 'менее 75 000'
    if income >= 75000 and income < 150000:
        return 'от 75 000 до 150 000'
    if income >= 150000 and income < 750000:
        return 'от 150 000 до 750 000'
    if income >= 750000:
        return '750 000 и более'
df['income_group'] = df['total_income'].apply(income_category)
df['income_group'].value_counts()

жилье и недвижимость    10811
автомобиль               4306
образование              4013
свадьба                  2324
Name: purpose_group, dtype: int64

2265604 - максимум 20667 - минимум, 167432 - среднее, 151888 - медиана.


от 150 000 до 750 000    10852
от 75 000 до 150 000      8688
менее 75 000              1865
750 000 и более             49
Name: income_group, dtype: int64

Данные по цели кредита распределены на 4 категории, больше всего кредитов взято с целью покупки или посторойки недвижимости, примерно поровну на автомобили и образование, меньше всего - на организацию свадеб.
Разделив данные на категории по доходу, получилось 4 категории:  1) до 75 000; 2) от 75 000 до 150 000; 3) от 150 000 до 750 000; 4) 750 000 и более. Самой многочисленной оказалась категория "от 75 000 до 150 000", самой маленькой "750 000 и более".

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [9]:
children_debt = df[['children', 'debt']] #выделим фрагмент таблицы, содержащий столбцы 'children' и 'debt'
def have_children(child): # напишем функцию которая разделит строки в таблице на 2 группы: с детьми и без
    if child > 0:
        return 'yes'
    else:
        return 'no'
children_debt['have_children'] = children_debt['children'].apply(have_children) #применим её к выделенному фрагменту датасета
#напечатаем сводную таблицу по доле должников
pd.pivot_table(children_debt, index=["have_children"], values=['debt'], aggfunc='mean')



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,debt
have_children,
no,0.075438
yes,0.092082


**Вывод**

Получается так, что люди с детьми имеют задолженность по кредиту чаще, 9,2% против 7,5%. Зависимость небольшая, но есть.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [10]:
# выберем столбцы, содержащие информацию по наличию задолженности и о семейном положении
family_debt = df[['family_status', 'debt']] 
#напечатаем сводную таблицу по доле должников
pd.pivot_table(family_debt, index=["family_status"], values=['debt'], aggfunc='mean').sort_values(by='debt', ascending=False)


,debt
family_status,
Не женат / не замужем,0.097509
гражданский брак,0.093471
женат / замужем,0.075452
в разводе,0.071130
вдовец / вдова,0.065693


Среди неженатых и проживающих совместно без регистрации должников больше, чем среди женатых, разведенных и вдовцов

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [11]:
# выберем столбцы, содержащие информацию о наличии задолженности и об уровне дохода
income_debt = df[['income_group', 'debt']]
#напечатаем сводную таблицу по доле должников
pd.pivot_table(income_debt, index=["income_group"], values=['debt'], aggfunc='mean').sort_values(by='debt', ascending=False)

,debt
income_group,
от 75 000 до 150 000,0.085635
750 000 и более,0.081633
от 150 000 до 750 000,0.078972
менее 75 000,0.072922


Это сложный вопрос, т.к. некоторые доходы были усреднены в процессе предобработки данных, однако процент должников почти не отличается в зависимости от уровня дохода. 

- Как разные цели кредита влияют на его возврат в срок?

In [12]:
# выберем столбцы, содержащие информацию о наличии задолженности и о цели кредита
purpose_debt = df[['purpose_group', 'debt']]
#напечатаем сводную таблицу по доле должников
pd.pivot_table(purpose_debt, index=["purpose_group"], values=['debt'], aggfunc='mean').sort_values(by='debt', ascending=False)

,debt
purpose_group,
автомобиль,0.093590
образование,0.092200
свадьба,0.080034
жилье и недвижимость,0.072334


Если верить данным сводной таблицы, можно предположить, что больше всего должников среди тех, кто берет кредит на покупку автомобиля, меньше всего - на покупку и постройку недвижимости

## Шаг 4. Общий вывод

Исходя из данных, представленных в таблице, можно сделать вывод о том, что чаще всего имеют задолженность по кредиту люди с детьми, холостые или состоящие в граждаском браке, берущие кредит на покупку автомобиля или на получение образования, наличие задолженности по кредиту не имеет явной связи с уровнем дохода. Однако имеющихся в моем арсенале способов работы с данными в pandas явно недостаточно, чтобы полноценно проанализировать представленные данные.

<div class="alert alert-info"> <b>Комментарий студента:
Спасибо! На самом деле долго мучил вопрос о критериях достоверности разницы в аналитике, например в науке используется р-критерий, который высчитывается исходя из объема выборки и типа данных. Мы его не использовали, потому что ещё не проходили в курсе или в аналитике им не пользуются?


<div class="alert alert-success">
<b>Комментарий ревьюера:</b>
    
Если ты имеешь в виду `p-value`, то его скоро тоже будем проходить, и он используется также в аналитке)